<a href="https://colab.research.google.com/github/vieduy/CS114.K21/blob/master/Benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"duyhoangvien","key":"0952d645a4b4335830e2e94ecdd947d0"}'}

In [2]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 68 Jun 24 07:51 kaggle.json


In [3]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [4]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
# List available datasets.
!kaggle datasets list

In [5]:
!kaggle datasets download -d meowmeowmeowmeowmeow/gtsrb-german-traffic-sign --force

 98% 601M/612M [00:16<00:00, 28.6MB/s]
100% 612M/612M [00:16<00:00, 38.6MB/s]


In [ ]:
!unzip gtsrb-german-traffic-sign.zip

In [1]:
cd drive/My\ Drive

/content/drive/My Drive


In [2]:
import matplotlib

# import the necessary packages
from scripts.preprocessing.imagetoarraypreprocessor import ImageToArrayPreprocessor
from scripts.preprocessing.simplepreprocessor import SimplePreprocessor
from scripts.preprocessing.patchpreprocessor import PatchPreprocessor
from scripts.preprocessing.meanpreprocessor import MeanPreprocessor
from scripts.callbacks.trainingmonitor import TrainingMonitor
from scripts.io.hdf5datasetgenerator import HDF5DatasetGenerator
from scripts.preprocessing.aspectawarepreprocessor import AspectAwarePreprocessor
from scripts.cnn.alexnet import AlexNet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import LabelEncoder
from scripts.io.simpledatasetloader import SimpleDatasetLoader
import json
import os
from imutils import paths
import random
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
aap = AspectAwarePreprocessor(64, 64)
iap = ImageToArrayPreprocessor()

In [ ]:
imagePaths = list(paths.list_images('../../Train'))
random.shuffle(imagePaths)
trainPaths = imagePaths[:31368]
testPaths = imagePaths[31368:]
length_train = len(trainPaths)
length_test = len(testPaths)
sdl = SimpleDatasetLoader(preprocessors=[])
(trainX, trainY) = sdl.load(trainPaths, verbose=500, length=length_train, name='../../train.hdf5')
(testX, testY) = sdl.load(testPaths, verbose=500, length=length_test, name='../../val.hdf5')

In [7]:
from sklearn.preprocessing import LabelBinarizer

# convert the labels from integers to vectors
trainX = trainX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)

In [5]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../train.hdf5', 'r')
trainX = np.array(db["images"])
trainY = np.array(db["labels"])

In [6]:
db_val = h5py.File('../../val.hdf5', 'r')
testX = np.array(db["images"])
testY = np.array(db["labels"])

In [ ]:
def step_decay(epoch):
  # initialize the base initial learning rate, drop factor, and
  # epochs to drop every
  initAlpha = 5e-4
  factor = 0.5
  dropEvery = 5

  # compute learning rate for the current epoch
  alpha = initAlpha * (factor ** np.floor((1 + epoch) / dropEvery))
  # return the learning rate
  return float(alpha)

In [3]:
# initialize the optimizer and model
from scripts.cnn.alexnet import AlexNet
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator

print("[INFO] compiling model...")
opt = Adam(lr=1e-4, decay=1e-4 / (30 * 0.5))
model = AlexNet.build(width=64, height=64, depth=3,
  classes=43, reg=0.0002)
model.compile(loss="categorical_crossentropy", optimizer=opt,
  metrics=["accuracy"])

aug = ImageDataGenerator(
	rotation_range=10,
	zoom_range=0.15,
	width_shift_range=0.1,
	height_shift_range=0.1,
	shear_range=0.15,
	horizontal_flip=False,
	vertical_flip=False,
	fill_mode="nearest")

[INFO] compiling model...


In [11]:
from tensorflow.keras.callbacks import ModelCheckpoint

fname = os.path.sep.join(['output', "weights-{epoch:03d}-{val_loss:.4f}.hdf5"])
checkpoint = ModelCheckpoint(fname, monitor="val_loss", mode="min", save_best_only=True, verbose=1)
figPath = os.path.sep.join(['output', "{}.png".format(os.getpid())])
jsonPath = os.path.sep.join(['output', "{}.json".format(os.getpid())])
callbacks = [TrainingMonitor(figPath, jsonPath=jsonPath), checkpoint]

In [12]:
print("[INFO] training network...")
H = model.fit(
	aug.flow(trainX, trainY, batch_size=128),
	validation_data=(testX, testY),
	steps_per_epoch=trainX.shape[0] // 128,
	epochs=5, verbose=1, callbacks=callbacks)

[INFO] training network...
Epoch 1/5
245/245 [==============================] - ETA: 0s - loss: 0.2902 - accuracy: 0.9901
Epoch 00001: val_loss improved from inf to 0.25866, saving model to output/weights-001-0.2587.hdf5
245/245 [==============================] - 64s 262ms/step - loss: 0.2902 - accuracy: 0.9901 - val_loss: 0.2587 - val_accuracy: 0.9985
Epoch 2/5
245/245 [==============================] - ETA: 0s - loss: 0.2877 - accuracy: 0.9897
Epoch 00002: val_loss did not improve from 0.25866
245/245 [==============================] - 63s 258ms/step - loss: 0.2877 - accuracy: 0.9897 - val_loss: 0.2882 - val_accuracy: 0.9909
Epoch 3/5
245/245 [==============================] - ETA: 0s - loss: 0.2801 - accuracy: 0.9910
Epoch 00003: val_loss improved from 0.25866 to 0.25087, saving model to output/weights-003-0.2509.hdf5
245/245 [==============================] - 63s 258ms/step - loss: 0.2801 - accuracy: 0.9910 - val_loss: 0.2509 - val_accuracy: 0.9991
Epoch 4/5
245/245 [==============

In [13]:
model.save('../../alexnet.model', overwrite=True)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: ../../alexnet.model/assets


In [20]:
model.load_weights('output/weights-010-0.3021.hdf5')

In [ ]:
from tensorflow.keras.models import load_model

print("[INFO] loading model...")
model = load_model('../../alexnet_45.model')

In [16]:
from os.path import basename, splitext

imagePaths = list((paths.list_images('../../Test')))
new_list = [splitext(basename(x))[0] for x in imagePaths]
fin_list = list(zip(imagePaths, new_list))
fin_list = [x[0] for x in sorted(fin_list,key=lambda x: int(x[1]))]
print(fin_list[0])
length = len(imagePaths)
sdl = SimpleDatasetLoader(preprocessors=[])
data = sdl.load_test(fin_list, verbose=2000, length=length)
data = data.astype("float") / 255.0

../../Test/00000.png
[INFO] processed 2000/12630
[INFO] processed 4000/12630
[INFO] processed 6000/12630
[INFO] processed 8000/12630
[INFO] processed 10000/12630
[INFO] processed 12000/12630


In [5]:
from sklearn.preprocessing import LabelBinarizer
import h5py

db = h5py.File('../../test.hdf5', 'r')
testImages = np.array(db["images"])
testImages = testImages.astype('float') / 255.0

In [6]:
labelNames = open("../../signnames.csv").read().strip().split("\n")[1:]
labelNames = [l.split(",")[1] for l in labelNames]

In [21]:
predictions = model.predict(testImages ,batch_size=64)
pred = predictions.argmax(axis=1)

In [ ]:
predictions = model.predict(data ,batch_size=128)
pred = predictions.argmax(axis=1)

In [22]:
import pandas as pd

test_df = pd.read_csv('Test.csv')
labels = test_df['ClassId'].values
print(labels[1000], pred[1000])

17 17


In [23]:
from sklearn.metrics import classification_report

print(classification_report(labels,
  predictions.argmax(axis=1), target_names=labelNames))

                                                    precision    recall  f1-score   support

                              Speed limit (20km/h)       0.98      1.00      0.99        60
                              Speed limit (30km/h)       0.99      0.98      0.99       720
                              Speed limit (50km/h)       0.98      0.99      0.99       750
                              Speed limit (60km/h)       0.99      0.96      0.97       450
                              Speed limit (70km/h)       0.99      0.98      0.98       660
                              Speed limit (80km/h)       0.97      0.97      0.97       630
                       End of speed limit (80km/h)       0.92      0.90      0.91       150
                             Speed limit (100km/h)       0.99      0.98      0.98       450
                             Speed limit (120km/h)       0.98      0.99      0.98       450
                                        No passing       0.98      0.99      0.

In [24]:
 from sklearn.metrics import accuracy_score

 print(accuracy_score(labels, pred))

0.97458432304038


In [ ]:
import cv2

image = cv2.imread('../../40_1.jpg')
for p in [aap, iap]:
  image = p.preprocess(image)

test_data = np.array([image])
test_data = test_data.astype('float') / 255.0
predict = model.predict(test_data)
print(predict.argmax(axis=1))